This is the first part of the notebook we have used to build the model submitted for the final evaluation. Kaggle allows only runs which last less than 9 hourse, so we needed to split the training into 2 parts, otherwise we would have exceeded the maximum time. Notice that, since we used all the data for the training, we don't have a validation set here. Of course, previously we have run the same code considering also the valuation set.

In [ ]:
# Importing libraries
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
# Class Weights are needed to contrast the problem of class imbalance.

labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

num_of_images_training = []
for l in labels:
    path = '../input/full-dataset/Dataset/training' + '/' + l
    num_img_class_l = len(os.listdir(path))
    num_of_images_training.append(num_img_class_l)

tot_num_images = 0
for i in range(len(labels)):
    tot_num_images += num_of_images_training[i]
    
weights = []
for i in range(len(labels)):
    num = (1 / num_of_images_training[i])*(tot_num_images)/14.0 
    weights.append(num)
    
print(weights)

# Vector of weights we will use for the training

class_weights = {0: weights[0], 1: weights[1], 2: weights[2], 3: weights[3], 4: weights[4], 5: weights[5], 6: weights[6], 7: weights[7], 8: weights[8], 9: weights[9], 10: weights[10], 11: weights[11], 12: weights[12], 13: weights[13]}

for i in range(len(labels)):
    print(str(i) + ': {:.3f}'.format(weights[i]))

In [ ]:
# Data augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

train_data_gen_data_aug = ImageDataGenerator(rotation_range=30,
                                        height_shift_range=50,
                                        width_shift_range=50,
                                        zoom_range=0.3, # To better recognize both veins (zoom in) and edges (zoom out) of the leaves
                                        horizontal_flip=True,
                                        vertical_flip=True, # To recognize leaves when they're upside down
                                        fill_mode='constant') # To avoid multiple leaves in the same image
                                        

print("IDG OK")

In [ ]:
training_dir = '../input/full-dataset/Dataset/training'
validation_dir = '../input/full-dataset/Dataset/validation'

train_gen_data_aug = train_data_gen_data_aug.flow_from_directory(directory=training_dir,
                                                           target_size=(256,256),
                                                           color_mode='rgb',
                                                           classes=None, 
                                                           class_mode='categorical',
                                                           batch_size=64,
                                                           shuffle=True,
                                                           seed=seed)


The training is divided into 2 steps

1.   First Step: 20 epochs in which all the layers of ResNet50 are setted to "trainable = False". In this way we are able to train a little bit the dense layers that we have put after ResNet50 before the fine tuning. In this phase we use a larger learning rate (1e-3)
2.   Second Step: 100 epochs in which the last layers of ResNet50 are setted to "trainable = True". In this way we fine tune ResNet50 togheter with our dense layers. In this phase we use a smaller learning rate (1e-4)

Splitting the training into 2 steps, we can fine tune the last layers of ResNet50 having our dense layers trained a little bit.

In [ ]:
input_shape = (256, 256, 3)
epochs = 20
print("DEBUG")

In [ ]:
supernet = tfk.applications.ResNet50(
    include_top=False, # In a first moment, none of the layers of ResNet50 is trainable
    input_shape=(224,224,3),
    weights='imagenet')

supernet.trainable = False
supernet.summary()


In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

    exps_dir = os.path.join('./working1')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)

    now = datetime.now().strftime('%b%d_%H-%M-%S')

    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)
      
    callbacks = []

    # Model checkpoint 
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only= True) # True to save only the best epoch 
    callbacks.append(ckpt_callback)

    # Visualize Learning on Tensorboard
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)
      
    # By default shows losses and metrics for both training and validation
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
    callbacks.append(tb_callback)


    return callbacks

Notice that, using all data for the training, we don't use Early Stopping.

In [ ]:
supernet.trainable = False

inputs = tfk.Input(shape=(256,256,3))

x = tfkl.Resizing(224, 224, interpolation="bicubic")(inputs) # 224x224 is the input shape used by ResNet50
x = tfkl.GaussianNoise(stddev=0.1)(x) # We add some noise to generalize better
x = preprocess_input(x) # Preprocessing using the preprocessing function of ResNet50


x = supernet(x) # ResNet50

x = tfkl.MaxPooling2D( # Max pooling to reduce the number of parameters
    pool_size=(2, 2), strides=None, padding="valid", data_format=None)(x)

# Our dense layers

x = tfkl.Flatten(name='Flattening')(x) # Flattening
x = tfkl.Dropout(0.5, seed=seed)(x)
x = tfkl.Dense(
    128, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    64, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
    14, # 14 classes of leaves
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

model_aug_RN50_fine = tfk.Model(inputs=inputs, outputs=outputs, name='model_aug_RN50_fine')
model_aug_RN50_fine.summary()

In [ ]:
# First step of the training
model_aug_RN50_fine.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-3), metrics='accuracy')

In [ ]:
callbacks = create_folders_and_callbacks(model_name='model_aug_RN50_fine')

history = model_aug_RN50_fine.fit(
    x = train_gen_data_aug,
    batch_size = 64,
    epochs = epochs,
    callbacks = callbacks,
    class_weight = class_weights # Class weights computed by us
).history

In [ ]:
model_aug_RN50_fine.get_layer('resnet50').trainable = True
for i, layer in enumerate(model_aug_RN50_fine.get_layer('resnet50').layers[:158]): # Fine Tuning
    layer.trainable=False
for i, layer in enumerate(model_aug_RN50_fine.get_layer('resnet50').layers):
    print(i, layer.name, layer.trainable)


model_aug_RN50_fine.summary()

In [ ]:
# First part of the second step of the training
model_aug_RN50_fine.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [ ]:
callbacks = create_folders_and_callbacks(model_name='model_aug_RN50_fine')
epochs = 50

history = model_aug_RN50_fine.fit(
    x = train_gen_data_aug,
    batch_size = 64,
    epochs = epochs,
    callbacks = callbacks,
    class_weight = class_weights
).history

In [ ]:
model_aug_RN50_fine.save('./working2')

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
#plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
#plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()